8-1 순환 신경망(Recurrent Neural Netword,RNN)

RNN은 입력과 출력을 시퀀스 단위로 처리하는 시퀀스(Sequence)모델이다. \
시퀀스 모델들 중 가장 기본적인 인공 신경망 시퀀스 모델이다.

1. 순환 신경망(RNN)

신경망들이 은닉층에서 활성화 함수를 지나 오직 출력층의 방향으로 향하는걸 피드 포워드 신경망(FFNN)이라고 한다.\
반면에 RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고 있다.

RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 "셀"이라고 한다. \
이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 "메모리 셀" 또는 "RNN 셀" 이라고 한다.

메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값을 "은닉 상태"라고 한다. \
또한 RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용할 수 있다. \
ex) 일대다, 다대일, 다대다

1) 일대다 구조 - 하나의 입력에 대해서 여러개의 출력을 의미  ex. 사진의 제목을 출력하는 이미지 캡셔닝 작업\
2) 다대일 구조 - 단어 시퀀스에 대해서 하나의 출력을 하는 구조  ex. 감성분류, 스팸 메일 분류 등\
3) 다대다 구조 - 단어 시퀀스에 대해 여러개의 출력을 하는 구조  ex. 챗봇, 번역기, 태깅 작업 등

2. 케라스로 RNN 구현하기

In [ ]:
from tensorflow.keras.layers import SimpleRNN
model.add(SimpleRNN(hidden_units)) # 케라스로 RNN층을 추가하는 방법

In [ ]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

hidden_units = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 셀과 출력층으로 보내는 값의 크기와도 동일하다. \
timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 한다. 시점의 수 \
input_dim = 입력의 크기

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length = 2, input_dim = 10)와 동일
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


output_dim은 hidden_units의 값인 3이다. 이 경우 batch_size를 현 단계에서 알 수 없으므로 (None,3)으로 출력되었다.

In [4]:
# batch_size를 정의한 코드
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [6]:
# return_sequences 매개 변수에 True값을 기재하여 3D 텐서를 리턴
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences = True))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


3. 파이썬으로 RNN 구현하기 \
h(t) = tanh(W(x)X(t)+W(h)h(t-1)+b))

In [ ]:
# 아래의 코드는 pseudocode(가상코드)
hidden_state_t = 0 # 초기 은닉 상태를 0으로 초기화
for input_t in input length: # 각 시점마다 입력받음
    output_t = tanh(input_t, hidden_state_t)
    hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.

t시점의 은닉 상태를 hidden_state_t 라는 변수로 선언하였고, 입력 데이터의 길이를 input_length로 선언하였다.\
이 경우, 입력 데이터의 길이는 곧 총 시점의 수(timesteps)가 된다. 그리고 t 시점의 입력값을 input_t로 선언하였다.

이해를 위해, 실제로 동작되는 RNN코드를 구현해보려고 한다. 아래 코드는 2D 텐서를 입력으로 받지만 실제로 케라스에서는 3D 텐서를 입력으로 받는걸\
기억해야한다. \
\
timesteps은 시점의 수이다(자연어 처리에서는 보통 문장의 길이를 뜻한다.)\
input_dim은 입력의 차원이다(자연어 처리에서는 보통 단어 벡터의 차원이다.)\
hidden_units는 은닉 상태의 크기로 메로리 셀의 용량이다(초기 상태는 0의 값을 가지는 벡터)

In [9]:
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,))

print('초기 은닉 상태 :',hidden_state_t)

초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


8의 차원을 가지는 0의 값으로 구성된 벡터가 출력.

In [10]:
Wx = np.random.random((hidden_units, input_dim)) # (8,4) 크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_units, hidden_units)) # (8,8) 크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_units, )) # (8,) 크기의 1D 텐서 생성. 이 값은 편향(bias)

print('가중치 Wx의 크기(shape) :',np.shape(Wx))
print('가중치 Wh의 크기(shape) :',np.shape(Wh))
print('편향의 크기(bias) :',np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(bias) : (8,)


In [11]:
total_hidden_states = []

# 각 시점 별 입력값.
for input_t in inputs :
    
    # Wx * Xt + Wh * Ht-1 + b(bias)
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh,hidden_state_t) + b)
    
    # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
    # 각 시점의 은닉 상태의 값을 계속해서 누적
    total_hidden_states.append(list(output_t))
    hidden_state_t = output_t
    
# 출력시 값을 깔끔하게 해주는 용도
total_hidden_states = np.stack(total_hidden_states, axis = 0)

# (timesteps, output_dim)
print('모든 시점의 은닉 상태 :')
print(total_hidden_states)

모든 시점의 은닉 상태 :
[[0.95749407 0.61523211 0.77539398 0.97269181 0.92629067 0.70294553
  0.95049057 0.80622658]
 [0.99988262 0.9998683  0.99995398 0.99990429 0.9999412  0.99995918
  0.99950515 0.99997959]
 [0.99998686 0.99998063 0.99999516 0.99998221 0.99998813 0.99999365
  0.99988754 0.99999517]
 [0.99998952 0.99997612 0.99999686 0.99998943 0.99999101 0.99999445
  0.99992076 0.9999967 ]
 [0.9999396  0.99995113 0.99998783 0.99987996 0.99995456 0.99998868
  0.99940525 0.99999229]
 [0.9999598  0.99996927 0.99999083 0.99993294 0.99997076 0.99999089
  0.9996726  0.99999313]
 [0.99997039 0.99994787 0.99999038 0.99992728 0.99994184 0.99998728
  0.99949849 0.99998669]
 [0.99991182 0.999933   0.99998062 0.99972594 0.99986938 0.99998246
  0.99864301 0.99998058]
 [0.99990782 0.99990682 0.99998606 0.99979    0.99988898 0.99998397
  0.99884133 0.99998728]
 [0.99997494 0.99995128 0.99999519 0.99996833 0.99997036 0.99999117
  0.99977262 0.99999377]]


4. 깊은 순환 신경망(Deep Recurrent Neural Network)

RNN도 다수의 은닉층을 가질 수 있다고 언급하였다. 순환 신경망(RNN)에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 \
보여준다.

In [12]:
# 은닉층을 2개 추가하는 경우의 코드
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length = 10, input_dim = 5, return_sequences=True)) # 이 부분
model.add(SimpleRNN(hidden_units, return_sequences=True))
# return_sequences = True 를 해주는것이 핵심(첫번째 은닉층은 다음 은닉층이 존재 하기 때문이다.)

5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)\
시점 t에서의 출력값을 예측할 때 이전 시점의 입력뿐만 아니라, 이후 시점의 입력 또한 예측에 기여할 수 있다는 아이디어에 기반

In [ ]:
from tensorflow.keras.layers import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape = (timesteps, input_dim)))

양방향 RNN도 다수의 은닉층을 가질 수 있다. 하지만 다른 모델도 마찬가지로 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아니다. \
은닉층을 추가하면 학습할 수 있는 양이 많아지지만 반대로 훈련 데이터 또한 많은 양이 필요하다.

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape =(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True)))

1.Embedding을 사용하며, 단어 집합의 크기가 5,000이고 임베딩 벡터의 차원은 100이다.\
2.은닉층에서는 SimpleRNN을 사용하며, 은닉 상태의 크기는 128입니다. \
3.훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.\
4.이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.\
5.은닉층은 1개입니다.

Embedding = 5,000(input(batch_size)) * 100(vector) = 500,000\
Wx = 100(embedding) * 128(hidden) = 12,800 \
Wh = 128 * 128 = 16.384\
bias(hidden) = 128\
Wy = 128\
bias(output) = 1\
total = 529,441